In [2]:
import numpy as np
import faiss
import time

In [3]:
keys = np.memmap("wmt14.state_dump.states.npy", dtype=np.float16, mode='r', shape=(133816080, 1024))
toy_keys = keys[:1000000].astype(np.float32)

In [4]:
vals = np.memmap("wmt14.state_dump.words.npy", dtype=np.int16, mode='r', shape=(133816080, 1))

In [40]:
index1 = faiss.index_factory(1024, "IVF16384_HNSW32,Flat")
index1 = faiss.index_cpu_to_all_gpus(index1)
index1.train(toy_keys)
index1.add(toy_keys)

RuntimeError: Error in virtual void* faiss::gpu::StandardGpuResourcesImpl::allocMemory(const faiss::gpu::AllocRequest&) at /project/faiss/faiss/gpu/StandardGpuResources.cpp:452: Error: 'err == cudaSuccess' failed: StandardGpuResources: alloc fail type TemporaryMemoryBuffer dev 0 space Device stream 0x564d025e7320 size 1610612736 bytes (cudaMalloc error out of memory [2])


In [37]:
k = 5
N = 1000
toy_test_keys = keys[1000000:1000000+N].astype(np.float32)

start = time.time()
dist, ids = index1.search(toy_test_keys, k)
print("elapsed time:", time.time() - start)

for nn in range(N):
    query_vocab_id = vals[1000000+nn][0]
    found_vocab_ids = [ vals[id][0].tolist() for id in ids[nn] ]
    print(query_vocab_id, found_vocab_ids)

elapsed time: 0.004383087158203125
5348 [5348, 5348, 5348, 5348, 5348]
11402 [23579, 4808, 4214, 7301, 201]
5 [5, 5, 5, 5, 5]
2 [2, 2, 2, 2, 2]
1954 [1735, 1954, 305, 2337, 1735]
286 [286, 286, 286, 286, 286]
1037 [1037, 1037, 1037, 1037, 1037]
30100 [30100, 30100, 3249, 45, 4]
4 [4, 4, 4, 4, 4]
6597 [6597, 20320, 20320, 1599, 10182]
4557 [4557, 4557, 4557, 4557, 4557]
19 [19, 19, 19, 19, 19]
6 [6, 6, 6, 6, 6]
14 [40, 6, 228, 6, 6]
2385 [362, 826, 17832, 25505, 5161]
8 [65, 65, 36, 6, 14]
6 [6, 14, 6, 40, 6]
3499 [3499, 10506, 19560, 10305, 10995]
322 [322, 322, 322, 322, 322]
2820 [2820, 2820, 2820, 2820, 2820]
3424 [21675, 4674, 26521, 803, 25650]
129 [30100, 1033, 357, 14, 381]
520 [520, 1788, 11607, 4557, 8222]
4557 [520, 4557, 248, 45, 1788]
4 [4, 4, 4, 4, 4]
520 [520, 1788, 4557, 8222, 11607]
4557 [4557, 520, 248, 45, 311]
186 [186, 186, 186, 186, 186]
12 [12, 12, 12, 12, 12]
1834 [1834, 1834, 1834, 1834, 1834]
1153 [1153, 1153, 248, 1153, 248]
5 [5, 5, 5, 5, 5]
2 [2, 2, 2, 2, 2]

In [5]:
index2 = faiss.index_factory(1024, "IVF16384_HNSW32,PQ32")
index2 = faiss.index_cpu_to_all_gpus(index2)
index2.train(toy_keys)
index2.add(toy_keys)

In [6]:
k = 5
N = 1000
toy_test_keys = keys[1000000:1000000+N].astype(np.float32)

start = time.time()
dist, ids = index2.search(toy_test_keys, k)
print("elapsed time:", time.time() - start)

for nn in range(N):
    query_vocab_id = vals[1000000+nn][0]
    found_vocab_ids = [ vals[id][0].tolist() for id in ids[nn] ]
    print(query_vocab_id, found_vocab_ids)

elapsed time: 0.00439453125
5348 [5348, 5348, 5348, 5348, 5348]
11402 [23579, 981, 7301, 4808, 7482]
5 [5, 5, 5, 5, 5]
2 [2, 2, 2, 2, 2]
1954 [1735, 1954, 608, 506, 1735]
286 [286, 286, 286, 286, 286]
1037 [1037, 1037, 1037, 1037, 1037]
30100 [30100, 30100, 4, 3249, 66]
4 [4, 4, 4, 4, 4]
6597 [6597, 20320, 20320, 10732, 10182]
4557 [4557, 4557, 4557, 4557, 4557]
19 [19, 19, 19, 19, 19]
6 [6, 6, 6, 6, 6]
14 [40, 6, 228, 6, 6]
2385 [146, 826, 17832, 362, 7389]
8 [65, 65, 264, 8, 14]
6 [14, 6, 6, 6, 8732]
3499 [3499, 10995, 10995, 10506, 271]
322 [322, 322, 322, 322, 322]
2820 [2820, 2820, 2820, 2820, 2820]
3424 [981, 25650, 26521, 4674, 6723]
129 [14, 1033, 30100, 357, 381]
520 [520, 8222, 1788, 4557, 11353]
4557 [520, 248, 4557, 45, 14335]
4 [4, 4, 4, 4, 4]
520 [520, 1788, 8222, 4557, 25064]
4557 [14335, 248, 520, 4557, 186]
186 [186, 186, 186, 186, 186]
12 [12, 12, 12, 12, 12]
1834 [1834, 1834, 1834, 1834, 1834]
1153 [1153, 248, 248, 1153, 248]
5 [5, 5, 5, 5, 5]
2 [2, 2, 2, 2, 2]
518 [

In [7]:
index2_cpu = faiss.index_gpu_to_cpu(index2)
faiss.write_index(index2_cpu, "wmt14_1m_ivf16384_hnsw32_pq32.index")

In [4]:
from random import randint

lfsr_roots = [
  [2, 1],
  [3, 2],
  [4, 3],
  [5, 3],
  [6, 5],
  [7, 6],
  [8, 6, 5, 4],
  [9, 5],
  [10, 7],
  [11, 9],
  [12, 6, 4, 1],
  [13, 4, 3, 1],
  [14, 5, 3, 1],
  [15, 14],
  [16, 15, 13, 4],
  [17, 14],
  [18, 11],
  [19, 6, 2, 1],
  [20, 17],
  [21, 19],
  [22, 21],
  [23, 18],
  [24, 23, 22, 17],
  [25, 22],
  [26, 6, 2, 1],
  [27, 5, 2, 1],
  [28, 25],
  [29, 27],
  [30, 6, 4, 1],
  [31, 28],
  [32, 22, 2, 1],
  [33, 20],
  [34, 27, 2, 1],
  [35, 33],
  [36, 25],
  [37, 5, 4, 3, 2, 1],
  [38, 6, 5, 1],
  [39, 35],
  [40, 38, 21, 19],
  [41, 38],
  [42, 41, 20, 19],
  [43, 42, 38, 37],
  [44, 43, 18, 17],
  [45, 44, 42, 41],
  [46, 45, 26, 25],
  [47, 42],
  [48, 47, 21, 20],
]

# code copied from http://www.christopia.net/blog/lazy-shuffled-list-generator
# credit goes to Christopher J. MacLellan
def srange(start, stop=None, step=1):
  """
  This generates the full range and shuffles it using a Fibonacci linear
  feedback shift register:
      https://en.wikipedia.org/wiki/Linear_feedback_shift_register#Fibonacci_LFSRs
  Here I use a table of precomputed primitive roots of different polynomials
  mod 2. In many ways this is similar to the multiplicative congruential
  generator in that we are iterating through elements of a finite field. We
  need primitive roots so that we can be sure we generate all elements in the
  range.  If we get elements outside the range we ignore them and continue
  iterating. Finally, we need the generator to be a primitive root of the
  selected modulus, so that we generate a full cycle. The seed provides the
  randomness for the permutation.
  This function has the same args as the builtin ``range'' function, but
  returns the values in shuffled order:
      range(stop)
      range(start, stop[, step])
  # >>> sorted([i for i in lazyshuffledrange3(10)])
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
  # >>> sorted([i for i in lazyshuffledrange3(2, 20, 3)])
  [2, 5, 8, 11, 14, 17]
  """
  if stop is None:
    stop = start
    start = 0
  l = (stop - start) // step
  nbits = l.bit_length()
  roots = lfsr_roots[nbits - 2]
  seed = randint(1, l)
  lfsr = seed
  while (True):
    if lfsr <= l:
      yield step * (lfsr - 1) + start
    bit = 0
    for r in roots:
      bit = (bit ^ (lfsr >> (nbits - r)))
    bit = (bit & 1)
    lfsr = (lfsr >> 1) | (bit << (nbits - 1))
    if lfsr == seed:
      break

In [5]:
np.array([ num for num, _ in zip(srange(0, 100), range(10)) ])

array([64, 95, 47, 23, 11,  5, 66, 32, 79, 39])

In [5]:
def built_index_from_dump(keys: np.array, dim: int, signature: str, train_data_size: int = 1000000):
    # store_size = 133816080
    # store_size = 10000000
    store_size = 133810000
    
    index = faiss.index_factory(dim, signature)
    index = faiss.index_cpu_to_all_gpus(index)
    print("index moved to gpu")
    
    # train_sample_idx = np.random.choice(np.arange(store_size), size=[min(train_data_size, store_size)], replace=False)
    train_sample_idx = np.array(sorted([ num for num, _ in zip(srange(0, store_size), range(train_data_size)) ]))
    train_sample = keys[train_sample_idx].astype(np.float32)
    # train_sample = keys[:train_data_size].astype(np.float32)
    print("sample generated")
    index.train(train_sample)
    print("training finished")
    
    block_size = 1000000
    n_blocks = store_size // block_size
    for i in range(n_blocks):
        print("adding block no.{0}".format(i))
        start = block_size * i
        end = block_size * (i + 1)
        index.add(keys[start:end].astype(np.float32))
    
    if block_size * n_blocks < store_size:
        start = block_size * n_blocks
        index.add(keys[start:store_size].astype(np.float32))
        
    return index

In [6]:
big_index = built_index_from_dump(keys, 1024, "OPQ64_128,IVF65536_HNSW32,PQ32", 4000000)

index moved to gpu
sample generated
training finished
adding block no.0
adding block no.1
adding block no.2
adding block no.3
adding block no.4
adding block no.5
adding block no.6
adding block no.7
adding block no.8
adding block no.9
adding block no.10
adding block no.11
adding block no.12
adding block no.13
adding block no.14
adding block no.15
adding block no.16
adding block no.17
adding block no.18
adding block no.19
adding block no.20
adding block no.21
adding block no.22
adding block no.23
adding block no.24
adding block no.25
adding block no.26
adding block no.27
adding block no.28
adding block no.29
adding block no.35
adding block no.36
adding block no.37
adding block no.38
adding block no.39
adding block no.40
adding block no.41
adding block no.42
adding block no.43
adding block no.44
adding block no.45
adding block no.46
adding block no.47
adding block no.48
adding block no.49
adding block no.50
adding block no.51
adding block no.52
adding block no.53
adding block no.54
adding

In [7]:
big_index_cpu = faiss.index_gpu_to_cpu(big_index)
faiss.write_index(big_index_cpu, "wmt14_opq64_128_ivf65536_hnsw32_pq32.index")

In [10]:
del big_index, big_index_cpu

In [5]:
big_index = faiss.read_index("wmt14_opq64_128_ivf65536_hnsw32_pq32.index")

In [12]:
k = 5
N = 6000
toy_test_keys = keys[133810000:].astype(np.float32)

start = time.time()
dist, ids = big_index.search(toy_test_keys, k)
print("elapsed time:", time.time() - start)

for nn in range(N):
    query_vocab_id = vals[133810000+nn][0]
    found_vocab_ids = [ vals[id][0].tolist() for id in ids[nn] ]
    print(query_vocab_id, found_vocab_ids)

elapsed time: 0.7420637607574463
5031 [5031, 57, 57, 57, 246]
4995 [4995, 4995, 4995, 4995, 4995]
3713 [11118, 11118, 5809, 1411, 863]
17 [17, 17, 17, 17, 17]
6 [6, 6, 6, 6, 6]
3915 [3915, 3915, 3915, 3915, 3915]
2477 [2477, 11577, 1555, 2477, 5917]
19 [19, 19, 19, 19, 19]
4338 [12540, 4338, 4338, 14546, 4338]
1044 [1044, 1044, 1044, 1044, 1044]
266 [266, 1568, 408, 266, 93]
60 [60, 60, 60, 60, 60]
6679 [6679, 6679, 6679, 6679, 6679]
6140 [325, 177, 4371, 408, 15700]
89 [89, 89, 89, 89, 89]
2 [2, 2, 2, 2, 2]
385 [385, 385, 385, 385, 385]
704 [704, 704, 704, 7108, 704]
100 [100, 100, 100, 100, 100]
148 [148, 148, 148, 148, 148]
6850 [1116, 19553, 240, 1692, 1116]
1662 [702, 702, 2132, 941, 702]
17009 [17009, 17009, 17009, 17009, 17009]
1038 [10987, 168, 29, 2640, 1038]
4 [4, 4, 4, 4, 4]
6 [6, 6, 6, 6, 6]
304 [304, 304, 2132, 304, 304]
100 [100, 100, 100, 100, 100]
1393 [2637, 1393, 2637, 2637, 3014]
69 [69, 69, 69, 69, 69]
7954 [7954, 7954, 7954, 7954, 7954]
23 [12, 23, 23, 23, 413]
155

8 [8, 8, 8, 8, 8]
16129 [860, 16586, 1937, 2102, 2474]
64 [64, 64, 64, 64, 64]
40 [40, 40, 40, 40, 40]
3400 [3400, 3400, 3796, 3400, 3400]
2099 [1652, 630, 1652, 216, 647]
4 [4, 4, 4, 4, 4]
1894 [1894, 1894, 1894, 1894, 1894]
36 [36, 36, 36, 36, 36]
21094 [21094, 21094, 21094, 21094, 21094]
7042 [7042, 7042, 7042, 7042, 16177]
12 [12, 12, 12, 12, 12]
6 [6, 6, 6, 6, 6]
24716 [24716, 25476, 25476, 25476, 25476]
765 [13226, 1808, 4986, 2650, 765]
4411 [4411, 4411, 4411, 4411, 4411]
15 [15, 15, 15, 15, 15]
5036 [765, 1845, 8066, 3610, 5155]
1122 [7901, 1647, 4096, 1647, 1122]
4 [4, 4, 4, 4, 4]
9 [9, 9, 9, 9, 9]
3978 [5781, 336, 840, 4194, 336]
36 [36, 36, 36, 36, 36]
14 [14, 14, 14, 14, 14]
5900 [5900, 5900, 5900, 5900, 5900]
1667 [1667, 1667, 1667, 1667, 1667]
387 [387, 387, 387, 387, 387]
413 [413, 413, 413, 413, 413]
189 [189, 189, 189, 189, 189]
22046 [22046, 22046, 22046, 22046, 22046]
378 [3996, 23057, 5721, 396, 4727]
8 [8, 8, 8, 8, 8]
829 [829, 829, 829, 829, 829]
15369 [15369, 153

7064 [7064, 13663, 1733, 1786, 5246]
1750 [1750, 1750, 1750, 1750, 1750]
1155 [6529, 2412, 5796, 1720, 223]
9 [9, 9, 9, 9, 9]
12 [12, 12, 12, 12, 12]
2191 [2191, 2191, 2191, 2191, 2191]
2707 [3897, 197, 197, 5428, 2707]
119 [119, 119, 119, 119, 119]
79 [79, 79, 79, 79, 79]
4315 [4315, 10374, 4315, 2314, 15258]
8 [8, 8, 8, 8, 8]
14 [14, 14, 14, 14, 14]
7839 [350, 197, 906, 7839, 350]
8 [8, 8, 8, 8, 8]
1687 [1687, 1687, 1687, 1687, 1687]
9 [9, 9, 9, 9, 9]
10067 [10067, 710, 10067, 1692, 10067]
5 [5, 5, 5, 5, 5]
2 [2, 2, 2, 2, 2]
1308 [1308, 1308, 1308, 1308, 1308]
677 [677, 677, 677, 677, 677]
6098 [12729, 6950, 298, 5951, 298]
590 [590, 590, 590, 590, 590]
5036 [5036, 5036, 5036, 5036, 5036]
600 [600, 600, 600, 600, 600]
875 [875, 1184, 1258, 823, 1358]
4 [4, 4, 4, 4, 4]
203 [203, 203, 203, 203, 203]
196 [196, 196, 196, 196, 196]
6166 [6166, 6166, 6166, 6166, 6166]
995 [995, 995, 995, 995, 995]
480 [480, 12501, 12501, 12501, 2695]
176 [5893, 22, 494, 4560, 12246]
119 [119, 119, 119, 119

7422 [235, 221, 424, 75, 45]
5160 [18457, 1821, 2601, 22453, 1395]
111 [4, 4, 4, 4, 4]
14 [235, 70, 14, 14, 37]
23997 [1809, 1845, 9978, 810, 40]
15269 [222, 984, 17628, 19278, 14855]
316 [316, 6997, 1903, 1442, 13594]
1039 [6363, 6358, 1966, 63, 45]
246 [17, 45, 17, 278, 17]
11000 [21459, 11963, 4304, 1001, 22514]
12409 [1406, 9978, 14508, 810, 1845]
837 [1692, 2696, 496, 5056, 304]
302 [1610, 1054, 23, 159, 61]
1026 [149, 367, 1231, 427, 2601]
317 [1231, 367, 11635, 2601, 427]
5 [5, 5, 5, 5, 5]
2 [2, 2, 2, 2, 2]
5979 [566, 566, 566, 5979, 5979]
4 [4, 4, 4, 4, 4]
732 [732, 732, 732, 732, 732]
437 [437, 437, 437, 255, 437]
26817 [743, 16044, 9309, 1022, 25861]
119 [119, 119, 119, 119, 119]
79 [79, 79, 79, 79, 79]
15269 [15269, 15269, 15269, 15269, 15269]
316 [316, 316, 316, 316, 316]
1039 [584, 11756, 1095, 1975, 1665]
6926 [3821, 1235, 8908, 5403, 5721]
10886 [10886, 10886, 10886, 10886, 10886]
2191 [2191, 2191, 2191, 2191, 2191]
2707 [6568, 4269, 197, 16129, 1718]
119 [119, 119, 119,

In [6]:
def precision_k(k, vals, ids, base):
    hit = 0
    for nn in range(ids.shape[0]):
        query_vocab_id = vals[base+nn][0]
        found_vocab_ids = [ vals[id_][0].tolist() for id_ in ids[nn] ]
        
        hit += sum([ 1 if query_vocab_id == id else 0 for id in found_vocab_ids ])
    return hit / (k * ids.shape[0])

In [7]:
base = 133810000
N = 6000
test_keys = keys[base:base+N].astype(np.float32)
metrics_held = []
for k in range(10):
    dist, ids = big_index.search(test_keys, k+1)
    metrics_held.append(precision_k(k+1, vals, ids, base))

In [8]:
base = 0
N = 6000
test_keys = keys[base:base+N].astype(np.float32)
metrics_inc = []
for k in range(10):
    dist, ids = big_index.search(test_keys, k+1)
    metrics_inc.append(precision_k(k+1, vals, ids, base))

In [31]:
base = 100000000
N = 6000
test_keys = keys[base:base+N].astype(np.float32)
for k in range(10):
    dist, ids = big_index.search(test_keys, k+1)
    print(precision_k(k+1, vals, ids, base))

1.0
0.91975
0.8847222222222222
0.86225
0.8466666666666667
0.8354166666666667
0.825952380952381
0.8190625
0.8124629629629629
0.807


In [1]:
from vega_datasets import data

data.stocks()

,symbol,date,price
0,MSFT,2000-01-01,39.81
1,MSFT,2000-02-01,36.35
2,MSFT,2000-03-01,43.22
3,MSFT,2000-04-01,28.37
4,MSFT,2000-05-01,25.45
...,...,...,...
555,AAPL,2009-11-01,199.91
556,AAPL,2009-12-01,210.73
557,AAPL,2010-01-01,192.06
558,AAPL,2010-02-01,204.62


In [14]:
import pandas as pd
import altair as alt

data = {'k': list(range(1,11)) * 2, 'test set': ['indexed'] * 10 + ['unindexed'] * 10, 'precision_k': metrics_inc + metrics_held}
source = pd.DataFrame(data=data)

In [15]:
selection = alt.selection_interval(bind='scales')

alt.Chart(source).mark_line(point=True).encode(
    x='k',
    y='precision_k',
    color='test set'
).add_selection(selection)

alt.Chart(...)